<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/NLP_with_LLMs_Fine_tuning_Models_for_Classification_%26_Question_Answering_14_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification FineTuning

In [ ]:
!pip install transformers datasets evaluate accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
columns = ["id", "country", "label", "text"]
tweets_data = pd.read_csv("/content/twitter_training.csv", names= columns)
tweets_data = tweets_data.sample(frac = 0.001)
print(tweets_data.shape)
tweets_data.head()

(75, 4)


,id,country,label,text
68433,3718,Cyberpunk2077,Positive,I can’t wait for this to work out
22509,4253,CS-GO,Irrelevant,"Another project done and dusted, check it out...."
21104,4013,CS-GO,Positive,"I'm going to play like shit xD, havent played ..."
63871,7745,MaddenNFL,Negative,So the @ EASPORTS _ MUT @ EAMaddenNFL servers ...
3834,1863,CallOfDutyBlackopsColdWar,Positive,The weapons in


In [ ]:
tweets_data.shape

(75, 4)

In [ ]:
tweets_data = tweets_data.drop(columns=["id", "country"], axis=1)
tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 68433 to 46955
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   75 non-null     object
 1   text    75 non-null     object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
tweets_data.dropna(inplace=True, axis = 0)
tweets_data = tweets_data.drop_duplicates()
tweets_data.shape

(75, 2)

In [ ]:
tweets_data["label"].value_counts()

,count
label,
Negative,25
Positive,22
Neutral,15
Irrelevant,13


In [ ]:
tweets_data["label"] = tweets_data["label"].replace({"Negative":0, "Neutral": 1, "Positive":2, "Irrelevant":3})
tweets_data.sample(10)

<ipython-input-40-5f887daefaf6>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tweets_data["label"] = tweets_data["label"].replace({"Negative":0, "Neutral": 1, "Positive":2, "Irrelevant":3})


,label,text
44607,1,verizon shut down we love that
21922,2,A Nice afternoon CS : GO hit play thanks to th...
24358,0,Attention with ARMYs looking back to purchase....
68433,2,I can’t wait for this to work out
23480,2,I live for this shit.
28898,0,was the same problem
62239,0,"Welp, GTA IV is still running like garbage in ..."
10916,2,Neat
19872,1,WOW DAY l IAMBRYCETV on all socials! . World...
49493,0,you


In [ ]:
from datasets import Dataset
tweets_ds = Dataset.from_pandas(tweets_data)
tweets_ds

Dataset({
    features: ['label', 'text', '__index_level_0__'],
    num_rows: 75
})

In [ ]:
tweets_ds = tweets_ds.remove_columns(["__index_level_0__"])
tweets_ds

Dataset({
    features: ['label', 'text'],
    num_rows: 75
})

In [ ]:
tweets_ds = tweets_ds.train_test_split(test_size=0.2)
tweets_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 60
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 15
    })
})

In [ ]:
tweets_ds["train"][5]

{'label': 3,
 'text': '26 percent average concurrent viewers on our first official live cast tonight. I am still speechless... But thank you to everyone who was pulled through here to support. com As for the big NiBBLE media relations team, you two guys have poured so much into bringing this all together. Yours It means more to me than you can imagine [UNK]'}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation = True)

In [ ]:
tokenizer("Worst thing to ever happen out in Gta history")

{'input_ids': [101, 5409, 2518, 2000, 2412, 4148, 2041, 1999, 14181, 2050, 2381, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.vocab_size

30522

In [ ]:
tokenized_tweets_ds = tweets_ds.map(preprocess_function, batched=True)
tokenized_tweets_ds

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 60
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 15
    })
})

In [ ]:
tweets_ds["train"][5]["text"]

'26 percent average concurrent viewers on our first official live cast tonight. I am still speechless... But thank you to everyone who was pulled through here to support. com As for the big NiBBLE media relations team, you two guys have poured so much into bringing this all together. Yours It means more to me than you can imagine [UNK]'

In [ ]:
tokenized_tweets_ds["train"][5]

{'label': 3,
 'text': '26 percent average concurrent viewers on our first official live cast tonight. I am still speechless... But thank you to everyone who was pulled through here to support. com As for the big NiBBLE media relations team, you two guys have poured so much into bringing this all together. Yours It means more to me than you can imagine [UNK]',
 'input_ids': [101,
  2656,
  3867,
  2779,
  16483,
  7193,
  2006,
  2256,
  2034,
  2880,
  2444,
  3459,
  3892,
  1012,
  1045,
  2572,
  2145,
  25146,
  1012,
  1012,
  1012,
  2021,
  4067,
  2017,
  2000,
  3071,
  2040,
  2001,
  2766,
  2083,
  2182,
  2000,
  2490,
  1012,
  4012,
  2004,
  2005,
  1996,
  2502,
  9152,
  11362,
  2865,
  4262,
  2136,
  1010,
  2017,
  2048,
  4364,
  2031,
  8542,
  2061,
  2172,
  2046,
  5026,
  2023,
  2035,
  2362,
  1012,
  6737,
  2009,
  2965,
  2062,
  2000,
  2033,
  2084,
  2017,
  2064,
  5674,
  100,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [ ]:
tokenizer.decode(tokenized_tweets_ds["train"][5]["input_ids"])

'[CLS] 26 percent average concurrent viewers on our first official live cast tonight. i am still speechless... but thank you to everyone who was pulled through here to support. com as for the big nibble media relations team, you two guys have poured so much into bringing this all together. yours it means more to me than you can imagine [UNK] [SEP]'

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0:"Negative", 1:"Neutral", 2:"Positive", 3:"Irrelevant"}
label2id = {"Negative":0, "Neutral": 1, "Positive":2, "Irrelevant":3}

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 2

batches_per_epoch = len(tokenized_tweets_ds["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_tweets_ds["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_validation_set = model.prepare_tf_dataset(
    tokenized_tweets_ds["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
model.fit(
    x = tf_train_set,
    validation_data = tf_validation_set,
    epochs = 2,
    callbacks = [metric_callback]
)

Epoch 1/2
3/3 [==============================] - 59s 12s/step - loss: 1.4028 - val_loss: 1.3809 - accuracy: 0.2667
Epoch 2/2
3/3 [==============================] - 26s 8s/step - loss: 1.3812 - val_loss: 1.3805 - accuracy: 0.4000


In [ ]:
# Save your trained TensorFlow model to a local directory
model.save_pretrained("my_classification_model")
tokenizer.save_pretrained('my_classification_model')

('my_classification_model/tokenizer_config.json',
 'my_classification_model/special_tokens_map.json',
 'my_classification_model/vocab.txt',
 'my_classification_model/added_tokens.json',
 'my_classification_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained('my_classification_model')
tokenizer = AutoTokenizer.from_pretrained('my_classification_model')

Some layers from the model checkpoint at my_classification_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at my_classification_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import pipeline
text = "This is awful. I get that profit-wise it was less than expected due to a huge budget"
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(text)

[{'label': 'Irrelevant', 'score': 0.25488537549972534}]

In [ ]:
from transformers import pipeline
text = "This is awful. I get that profit-wise it was less than expected due to a huge budget"
tokenizer = AutoTokenizer.from_pretrained("my_classification_model")
inputs = tokenizer(text, return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(1, 22), dtype=int32, numpy=
array([[ 101, 2023, 2003, 9643, 1012, 1045, 2131, 2008, 5618, 1011, 7968,
        2009, 2001, 2625, 2084, 3517, 2349, 2000, 1037, 4121, 5166,  102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 22), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("my_classification_model")
outputs = model(**inputs)
outputs

Some layers from the model checkpoint at my_classification_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at my_classification_model and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[ 0.00791983, -0.01937154,  0.00960617,  0.02536083]],
      dtype=float32)>, hidden_states=None, attentions=None)

In [ ]:
predicted_class_id = int(tf.math.argmax(outputs.logits, axis=-1)[0])
predicted_class_id

3

In [ ]:
model.config.id2label[predicted_class_id]

'Irrelevant'

#Token Classification / Named Entity Recognition

In [ ]:
!pip install transformers datasets evaluate seqeval accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=2a5ce7c1db206b39d255bb906d26394eb3e2bb513d6b5af162971a77b22a1760
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ner_data = pd.read_csv("/content/ner.csv", encoding="unicode_escape")
print(ner_data.shape)
ner_data = ner_data.sample(frac= 0.001)
print(ner_data.shape)
ner_data.head()

(47959, 4)
(48, 4)


,Sentence #,Sentence,POS,Tag
825,Sentence: 826,Insurgents in Iraq killed at least nine people...,"['NNS', 'IN', 'NNP', 'VBD', 'IN', 'JJS', 'CD',...","['O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', '..."
43958,Sentence: 43959,The U.N. tribunal says that during the violenc...,"['DT', 'NNP', 'NN', 'VBZ', 'IN', 'IN', 'DT', '...","['O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', '..."
37476,Sentence: 37477,Thousands of police and National Guard troops ...,"['NNS', 'IN', 'NN', 'CC', 'NNP', 'NNP', 'NNS',...","['O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O..."
8681,Sentence: 8682,Niger 's electoral commission says it is delay...,"['NNP', 'POS', 'JJ', 'NN', 'VBZ', 'PRP', 'VBZ'...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
46884,Sentence: 46885,A court spokesman says Klein has 10 days to ap...,"['DT', 'NN', 'NN', 'VBZ', 'NNP', 'VBZ', 'CD', ...","['O', 'O', 'O', 'O', 'B-per', 'O', 'B-tim', 'O..."


In [ ]:
ner_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 825 to 41815
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  48 non-null     object
 1   Sentence    48 non-null     object
 2   POS         48 non-null     object
 3   Tag         48 non-null     object
dtypes: object(4)
memory usage: 1.9+ KB


In [ ]:
ner_data.shape

(48, 4)

In [ ]:
import ast
def preprocessing_data(df):
    def parse_tags(tag_str):
        try:
            return [str(word) for word in ast.literal_eval(tag_str)]
        except (ValueError, SyntaxError):
            # Handle cases where the string cannot be evaluated
            return []

    # Apply the parsing function to the 'Tag' column
    df['Tag'] = df['Tag'].apply(parse_tags)

    return df

In [ ]:
ner_data = preprocessing_data(ner_data)

In [ ]:
ner_data.sample(10)


,Sentence #,Sentence,POS,Tag
16206,Sentence: 16207,At least five people have been killed and six ...,"['IN', 'JJS', 'CD', 'NNS', 'VBP', 'VBN', 'VBN'...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
564,Sentence: 565,U.S. health officials have called for updated ...,"['NNP', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'VBN'...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
5836,Sentence: 5837,"As the vote count continues , Iraqi officials ...","['IN', 'DT', 'NN', 'NN', 'VBZ', ',', 'JJ', 'NN...","[O, O, O, O, O, O, B-gpe, O, O, O, O, O, O, O,..."
31514,Sentence: 31515,The program involved shutting down Chinese mil...,"['DT', 'NN', 'VBN', 'VBG', 'RP', 'JJ', 'JJ', '...","[O, O, O, O, O, B-gpe, O, O, O, O, O, O, O, O,..."
37449,Sentence: 37450,The suit seeks an unspecified amount of money .,"['DT', 'NN', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'N...","[O, O, O, O, O, O, O, O, O]"
10282,Sentence: 10283,Qatar had been the only Gulf Arab state to hav...,"['NNP', 'VBD', 'VBN', 'DT', 'JJ', 'NNP', 'NNP'...","[B-geo, O, O, O, O, B-geo, I-geo, O, O, O, O, ..."
3076,Sentence: 3077,"Also Sunday , Sibghatullah Mujaddedi , who cha...","['RB', 'NNP', ',', 'NNP', 'NNP', ',', 'WP', 'V...","[O, B-tim, O, B-per, I-per, O, O, O, O, O, O, ..."
43958,Sentence: 43959,The U.N. tribunal says that during the violenc...,"['DT', 'NNP', 'NN', 'VBZ', 'IN', 'IN', 'DT', '...","[O, B-geo, O, O, O, O, O, O, O, O, B-tim, O, O..."
26751,Sentence: 26752,Two Americans were killed and four others woun...,"['CD', 'NNS', 'VBD', 'VBN', 'CC', 'CD', 'NNS',...","[O, B-gpe, O, O, O, O, O, O, O, O, O, O]"
41005,Sentence: 41006,The spokesman said Sunday that 41 of those arr...,"['DT', 'NN', 'VBD', 'NNP', 'IN', 'CD', 'IN', '...","[O, O, O, B-tim, O, O, O, O, O, O, O, O, O, O,..."


In [ ]:
ner_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 825 to 41815
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  48 non-null     object
 1   Sentence    48 non-null     object
 2   POS         48 non-null     object
 3   Tag         48 non-null     object
dtypes: object(4)
memory usage: 1.9+ KB


In [ ]:
from nltk.tokenize import word_tokenize

def tokenize_sentence(sentence):
  return word_tokenize(sentence)

ner_data["token"] = ner_data["Sentence"].apply(tokenize_sentence)
ner_data.sample(10)

,Sentence #,Sentence,POS,Tag,token
13740,Sentence: 13741,News agencies that obtained the report say it ...,"['NN', 'NNS', 'WDT', 'VBD', 'DT', 'NN', 'VBP',...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[News, agencies, that, obtained, the, report, ..."
28477,Sentence: 28478,"Kennedy , who has brain cancer , collapsed dur...","['NNP', ',', 'WP', 'VBZ', 'NN', 'NN', ',', 'VB...","[B-per, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[Kennedy, ,, who, has, brain, cancer, ,, colla..."
16170,Sentence: 16171,President Nursultan Nazarbayev has ruled the f...,"['NNP', 'NNP', 'NNP', 'VBZ', 'VBN', 'DT', 'JJ'...","[B-per, I-per, I-per, O, O, O, O, B-geo, I-geo...","[President, Nursultan, Nazarbayev, has, ruled,..."
18463,Sentence: 18464,"Uganda , which provides the bulk of the AU tro...","['NNP', ',', 'WDT', 'VBZ', 'DT', 'NN', 'IN', '...","[B-org, O, O, O, O, O, O, O, B-org, O, O, O, O...","[Uganda, ,, which, provides, the, bulk, of, th..."
43676,Sentence: 43677,In an interview published Saturday in the offi...,"['IN', 'DT', 'NN', 'VBN', 'NNP', 'IN', 'DT', '...","[O, O, O, O, B-tim, O, O, O, B-org, I-org, O, ...","[In, an, interview, published, Saturday, in, t..."
32328,Sentence: 32329,""" That is hemp seed he is sowing ; be careful ...","['``', 'DT', 'VBZ', 'JJ', 'NN', 'PRP', 'VBZ', ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[``, That, is, hemp, seed, he, is, sowing, ;, ..."
20864,Sentence: 20865,The document was filed late Wednesday with the...,"['DT', 'NN', 'VBD', 'VBN', 'JJ', 'NNP', 'IN', ...","[O, O, O, O, O, B-tim, O, O, B-org, I-org, I-o...","[The, document, was, filed, late, Wednesday, w..."
30763,Sentence: 30764,A provincial government spokesman says the fig...,"['DT', 'JJ', 'NN', 'NN', 'VBZ', 'DT', 'VBG', '...","[O, O, O, O, O, O, O, O, B-org, O, O, O, B-tim...","[A, provincial, government, spokesman, says, t..."
37559,Sentence: 37560,Venezuela 's election chief estimates voter tu...,"['NNP', 'POS', 'NN', 'NN', 'VBZ', 'NN', 'NN', ...","[B-geo, O, O, O, O, O, O, O, O, O, O, O]","[Venezuela, 's, election, chief, estimates, vo..."
564,Sentence: 565,U.S. health officials have called for updated ...,"['NNP', 'NN', 'NNS', 'VBP', 'VBN', 'IN', 'VBN'...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[U.S., health, officials, have, called, for, u..."


In [ ]:
length_column_tag = ner_data["Tag"].apply(lambda x: len(x))
length_column_token = ner_data["token"].apply(lambda x: len(x))

In [ ]:
length_column_tag.head()

,Tag
825,15
43958,37
37476,27
8681,19
46884,18


In [ ]:
length_column_token.head()

,token
825,15
43958,37
37476,27
8681,19
46884,18


In [ ]:
matching_lengths = length_column_tag == length_column_token
matching_lengths

,0
825,True
43958,True
37476,True
8681,True
46884,True
26751,True
24785,True
36172,True
28242,True
22760,True


In [ ]:
ner_data = ner_data[matching_lengths]
ner_data.shape

(48, 5)

In [ ]:
all_words = [ word for sublist in ner_data["Tag"].tolist() for word in sublist]
unique_words = sorted(set(all_words))
unique_words

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-org',
 'B-per',
 'B-tim',
 'I-eve',
 'I-geo',
 'I-org',
 'I-per',
 'O']

In [ ]:
tag_mapper = {tag: index for index, tag in enumerate(unique_words)}
tag_mapper

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-org': 4,
 'B-per': 5,
 'B-tim': 6,
 'I-eve': 7,
 'I-geo': 8,
 'I-org': 9,
 'I-per': 10,
 'O': 11}

In [ ]:
ner_data = ner_data.copy()
ner_data["Tag"] = ner_data["Tag"].apply(lambda x: [tag_mapper[word] for word in x])
ner_data.sample(10)

,Sentence #,Sentence,POS,Tag,token
825,Sentence: 826,Insurgents in Iraq killed at least nine people...,"['NNS', 'IN', 'NNP', 'VBD', 'IN', 'JJS', 'CD',...","[11, 11, 2, 11, 11, 11, 11, 11, 11, 11, 6, 11,...","[Insurgents, in, Iraq, killed, at, least, nine..."
13740,Sentence: 13741,News agencies that obtained the report say it ...,"['NN', 'NNS', 'WDT', 'VBD', 'DT', 'NN', 'VBP',...","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[News, agencies, that, obtained, the, report, ..."
37559,Sentence: 37560,Venezuela 's election chief estimates voter tu...,"['NNP', 'POS', 'NN', 'NN', 'VBZ', 'NN', 'NN', ...","[2, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[Venezuela, 's, election, chief, estimates, vo..."
3076,Sentence: 3077,"Also Sunday , Sibghatullah Mujaddedi , who cha...","['RB', 'NNP', ',', 'NNP', 'NNP', ',', 'WP', 'V...","[11, 6, 11, 5, 10, 11, 11, 11, 11, 11, 11, 11,...","[Also, Sunday, ,, Sibghatullah, Mujaddedi, ,, ..."
10282,Sentence: 10283,Qatar had been the only Gulf Arab state to hav...,"['NNP', 'VBD', 'VBN', 'DT', 'JJ', 'NNP', 'NNP'...","[2, 11, 11, 11, 11, 2, 8, 11, 11, 11, 11, 11, ...","[Qatar, had, been, the, only, Gulf, Arab, stat..."
473,Sentence: 474,Israeli military officials say troops had spot...,"['JJ', 'JJ', 'NNS', 'VBP', 'NNS', 'VBD', 'VBN'...","[3, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11...","[Israeli, military, officials, say, troops, ha..."
10663,Sentence: 10664,Ms. del Ponte said Monday Croatia is now fully...,"['NNP', 'NNP', 'NNP', 'VBD', 'NNP', 'NNP', 'VB...","[5, 10, 10, 11, 6, 2, 11, 11, 11, 11, 11, 11, ...","[Ms., del, Ponte, said, Monday, Croatia, is, n..."
16206,Sentence: 16207,At least five people have been killed and six ...,"['IN', 'JJS', 'CD', 'NNS', 'VBP', 'VBN', 'VBN'...","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[At, least, five, people, have, been, killed, ..."
41414,Sentence: 41415,"Last week , Mr. Putin ordered the special forc...","['JJ', 'NN', ',', 'NNP', 'NNP', 'VBD', 'DT', '...","[11, 11, 11, 5, 10, 11, 11, 11, 11, 11, 11, 11...","[Last, week, ,, Mr., Putin, ordered, the, spec..."
30763,Sentence: 30764,A provincial government spokesman says the fig...,"['DT', 'JJ', 'NN', 'NN', 'VBZ', 'DT', 'VBG', '...","[11, 11, 11, 11, 11, 11, 11, 11, 4, 11, 11, 11...","[A, provincial, government, spokesman, says, t..."


In [ ]:
ner_data.drop(columns=["Sentence #", "Sentence", "POS"], inplace=True)
ner_data.sample(10)

,Tag,token
33696,"[11, 11, 11, 11, 11, 2, 8, 11, 11, 4, 11, 11, ...","[Some, five, million, people, in, South, Afric..."
36172,"[3, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11...","[Turkish, officials, say, they, will, send, ne..."
24785,"[11, 11, 11, 11, 11, 2, 11, 11, 2, 11, 11, 11,...","[Both, candidates, opposed, independence, for,..."
32328,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[``, That, is, hemp, seed, he, is, sowing, ;, ..."
33040,"[11, 11, 11, 6, 11, 11, 11, 11, 11, 11, 11, 11...","[Those, talks, resumed, Sunday, with, generals..."
3144,"[11, 11, 11, 11, 11, 6, 11, 11, 11, 11, 11, 11...","[A, lengthy, report, released, late, Monday, s..."
37559,"[2, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11]","[Venezuela, 's, election, chief, estimates, vo..."
43958,"[11, 2, 11, 11, 11, 11, 11, 11, 11, 11, 6, 11,...","[The, U.N., tribunal, says, that, during, the,..."
29874,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[In, another, scenario, ,, terrorists, would, ..."
46884,"[11, 11, 11, 11, 5, 11, 6, 11, 11, 11, 11, 11,...","[A, court, spokesman, says, Klein, has, 10, da..."


In [ ]:
from datasets import Dataset
ner_ds = Dataset.from_pandas(ner_data)
ner_ds

Dataset({
    features: ['Tag', 'token', '__index_level_0__'],
    num_rows: 48
})

In [ ]:
ner_ds = ner_ds.train_test_split(test_size=0.2)
ner_ds

DatasetDict({
    train: Dataset({
        features: ['Tag', 'token', '__index_level_0__'],
        num_rows: 38
    })
    test: Dataset({
        features: ['Tag', 'token', '__index_level_0__'],
        num_rows: 10
    })
})

In [ ]:
example = ner_ds["train"][0]
example

{'Tag': [11, 11, 2, 11, 11, 11, 11, 11, 11, 11, 6, 11, 11, 2, 11],
 'token': ['Insurgents',
  'in',
  'Iraq',
  'killed',
  'at',
  'least',
  'nine',
  'people',
  'in',
  'attacks',
  'Sunday',
  'northeast',
  'of',
  'Baghdad',
  '.'],
 '__index_level_0__': 825}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
example = ner_ds["train"][0]
tokenized_input = tokenizer(example["token"], is_split_into_words=True)
print(tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'input_ids': [101, 20541, 1999, 5712, 2730, 2012, 2560, 3157, 2111, 1999, 4491, 4465, 4794, 1997, 13952, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['[CLS]',
 'insurgents',
 'in',
 'iraq',
 'killed',
 'at',
 'least',
 'nine',
 'people',
 'in',
 'attacks',
 'sunday',
 'northeast',
 'of',
 'baghdad',
 '.',
 '[SEP]']

In [ ]:
tokenized_input = tokenizer(example["token"], is_split_into_words=True)
print(tokenized_input)

{'input_ids': [101, 20541, 1999, 5712, 2730, 2012, 2560, 3157, 2111, 1999, 4491, 4465, 4794, 1997, 13952, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=True)

  labels = []
  for i, label in enumerate(examples[f"Tag"]):
    word_ids = tokenized_inputs.word_ids(batch_index=i)

    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)

  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [ ]:
tokenized_ner_ds = ner_ds.map(tokenize_and_align_labels, batched=True)
tokenized_ner_ds

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tag', 'token', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 38
    })
    test: Dataset({
        features: ['Tag', 'token', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})

In [ ]:
print(tokenized_ner_ds["train"][0]["Tag"])
print(tokenized_ner_ds["train"][0]["token"])
print(tokenized_ner_ds["train"][0]["labels"])

[11, 11, 2, 11, 11, 11, 11, 11, 11, 11, 6, 11, 11, 2, 11]
['Insurgents', 'in', 'Iraq', 'killed', 'at', 'least', 'nine', 'people', 'in', 'attacks', 'Sunday', 'northeast', 'of', 'Baghdad', '.']
[-100, 11, 11, 2, 11, 11, 11, 11, 11, 11, 11, 6, 11, 11, 2, 11, -100]


In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
import evaluate
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_predictions = [
      [unique_words[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [unique_words[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]

  results = seqeval.compute(predictions=true_predictions, references=true_labels)

  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

In [ ]:
tag_mapper

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-org': 4,
 'B-per': 5,
 'B-tim': 6,
 'I-eve': 7,
 'I-geo': 8,
 'I-org': 9,
 'I-per': 10,
 'O': 11}

In [ ]:
# Create id2label mapping by inverting the tag_mapper
id2label = {index: tag for tag, index in tag_mapper.items()}

# Display the id2label mapping
print(id2label)

{0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-org', 5: 'B-per', 6: 'B-tim', 7: 'I-eve', 8: 'I-geo', 9: 'I-org', 10: 'I-per', 11: 'O'}


In [ ]:
label2id = {tag: index for index, tag in id2label.items()}
label2id

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-org': 4,
 'B-per': 5,
 'B-tim': 6,
 'I-eve': 7,
 'I-geo': 8,
 'I-org': 9,
 'I-per': 10,
 'O': 11}

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 2

batches_per_epoch = len(tokenized_ner_ds["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0,
                                       weight_decay_rate=0.01,
                                       num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForTokenClassification
model = TFAutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(unique_words), id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [ ]:
len(unique_words)

12

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_ner_ds["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_validation_set = model.prepare_tf_dataset(
    tokenized_ner_ds["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf
model.compile(optimizer=optimizer)
model.summary()

Model: "tf_distil_bert_for_token_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 dropout_99 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  9228      
                                                                 
Total params: 66372108 (253.19 MB)
Trainable params: 66372108 (253.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
model.fit(
    x = tf_train_set,
    validation_data = tf_validation_set,
    epochs = 2,
    callbacks = [metric_callback]
)

Epoch 1/2
2/2 [==============================] - 62s 10s/step - loss: 2.5213 - val_loss: 2.5134 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1: 0.0000e+00 - accuracy: 0.0367


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/2
2/2 [==============================] - 12s 7s/step - loss: 2.5141 - val_loss: 2.5134 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1: 0.0000e+00 - accuracy: 0.0367


In [ ]:
from transformers import pipeline
text = "Rome is a beutiful city in Italy"
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
classifier(text)

[{'entity': 'B-per',
  'score': 0.117241345,
  'index': 1,
  'word': 'rome',
  'start': 0,
  'end': 4},
 {'entity': 'B-per',
  'score': 0.09802871,
  'index': 2,
  'word': 'is',
  'start': 5,
  'end': 7},
 {'entity': 'B-art',
  'score': 0.09698679,
  'index': 3,
  'word': 'a',
  'start': 8,
  'end': 9},
 {'entity': 'B-per',
  'score': 0.11069658,
  'index': 4,
  'word': 'be',
  'start': 10,
  'end': 12},
 {'entity': 'I-eve',
  'score': 0.105907984,
  'index': 5,
  'word': '##uti',
  'start': 12,
  'end': 15},
 {'entity': 'I-eve',
  'score': 0.10389044,
  'index': 6,
  'word': '##ful',
  'start': 15,
  'end': 18},
 {'entity': 'B-art',
  'score': 0.10762273,
  'index': 7,
  'word': 'city',
  'start': 19,
  'end': 23},
 {'entity': 'I-eve',
  'score': 0.10700349,
  'index': 8,
  'word': 'in',
  'start': 24,
  'end': 26},
 {'entity': 'B-art',
  'score': 0.10815987,
  'index': 9,
  'word': 'italy',
  'start': 27,
  'end': 32}]

#Question Answering

In [39]:
!pip install transformers datasets evaluate accelerate

In [135]:
from datasets import load_dataset
squad_ds = load_dataset("squad", split = "train[:500]")
squad_ds

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 500
})

In [136]:
squad_ds = squad_ds.train_test_split(test_size=0.2)
squad_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 400
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 100
    })
})

In [137]:
squad_ds["train"][5]

{'id': '5733a4c54776f41900660f2f',
 'title': 'University_of_Notre_Dame',
 'context': "The College of Science was established at the university in 1865 by president Father Patrick Dillon. Dillon's scientific courses were six years of work, including higher-level mathematics courses. Today the college, housed in the newly built Jordan Hall of Science, includes over 1,200 undergraduates in six departments of study – biology, chemistry, mathematics, physics, pre-professional studies, and applied and computational mathematics and statistics (ACMS) – each awarding Bachelor of Science (B.S.) degrees. According to university statistics, its science pre-professional program has one of the highest acceptance rates to medical school of any university in the United States.",
 'question': 'How many years long was a scientific course under Patrick Dillon at Notre Dame?',
 'answers': {'text': ['six years'], 'answer_start': [134]}}

In [138]:
squad_ds["train"][5]["context"]

"The College of Science was established at the university in 1865 by president Father Patrick Dillon. Dillon's scientific courses were six years of work, including higher-level mathematics courses. Today the college, housed in the newly built Jordan Hall of Science, includes over 1,200 undergraduates in six departments of study – biology, chemistry, mathematics, physics, pre-professional studies, and applied and computational mathematics and statistics (ACMS) – each awarding Bachelor of Science (B.S.) degrees. According to university statistics, its science pre-professional program has one of the highest acceptance rates to medical school of any university in the United States."

In [139]:
squad_ds["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [140]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.is_fast # Highly performant tokenizer implemented in Rust

True

In [141]:
context = squad_ds["train"][5]["context"]
question = squad_ds["train"][5]["question"]
question, context

('How many years long was a scientific course under Patrick Dillon at Notre Dame?',
 "The College of Science was established at the university in 1865 by president Father Patrick Dillon. Dillon's scientific courses were six years of work, including higher-level mathematics courses. Today the college, housed in the newly built Jordan Hall of Science, includes over 1,200 undergraduates in six departments of study – biology, chemistry, mathematics, physics, pre-professional studies, and applied and computational mathematics and statistics (ACMS) – each awarding Bachelor of Science (B.S.) degrees. According to university statistics, its science pre-professional program has one of the highest acceptance rates to medical school of any university in the United States.")

In [142]:
inputs = tokenizer(question, context)
inputs

{'input_ids': [101, 2129, 2116, 2086, 2146, 2001, 1037, 4045, 2607, 2104, 4754, 14602, 2012, 10289, 8214, 1029, 102, 1996, 2267, 1997, 2671, 2001, 2511, 2012, 1996, 2118, 1999, 6725, 2011, 2343, 2269, 4754, 14602, 1012, 14602, 1005, 1055, 4045, 5352, 2020, 2416, 2086, 1997, 2147, 1010, 2164, 3020, 1011, 2504, 5597, 5352, 1012, 2651, 1996, 2267, 1010, 7431, 1999, 1996, 4397, 2328, 5207, 2534, 1997, 2671, 1010, 2950, 2058, 1015, 1010, 3263, 8324, 2015, 1999, 2416, 7640, 1997, 2817, 1516, 7366, 1010, 6370, 1010, 5597, 1010, 5584, 1010, 3653, 1011, 2658, 2913, 1010, 1998, 4162, 1998, 15078, 5597, 1998, 6747, 1006, 9353, 5244, 1007, 1516, 2169, 21467, 5065, 1997, 2671, 1006, 1038, 1012, 1055, 1012, 1007, 5445, 1012, 2429, 2000, 2118, 6747, 1010, 2049, 2671, 3653, 1011, 2658, 2565, 2038, 2028, 1997, 1996, 3284, 9920, 6165, 2000, 2966, 2082, 1997, 2151, 2118, 1999, 1996, 2142, 2163, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [143]:
tokenizer.decode(inputs["input_ids"])

"[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the college of science was established at the university in 1865 by president father patrick dillon. dillon's scientific courses were six years of work, including higher - level mathematics courses. today the college, housed in the newly built jordan hall of science, includes over 1, 200 undergraduates in six departments of study – biology, chemistry, mathematics, physics, pre - professional studies, and applied and computational mathematics and statistics ( acms ) – each awarding bachelor of science ( b. s. ) degrees. according to university statistics, its science pre - professional program has one of the highest acceptance rates to medical school of any university in the united states. [SEP]"

In [144]:
squad_ds["train"][5]["answers"]

{'text': ['six years'], 'answer_start': [134]}

In [145]:
# first method but not efficient
inputs = tokenizer(
    question,
    context,
    max_length=30,
    truncation="only_second",

)
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS]
how
many
years
long
was
a
scientific
course
under
patrick
dillon
at
notre
dame
?
[SEP]
the
college
of
science
was
established
at
the
university
in
1865
by
[SEP]


In [146]:
# second method but efficient
inputs = tokenizer(
    question,
    context,
    max_length=30,
    truncation="only_second",
    return_overflowing_tokens=True,

)
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the college of science was established at the university in 1865 by [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] president father patrick dillon. dillon's scientific courses were six [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] years of work, including higher - level mathematics courses. today [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the college, housed in the newly built jordan hall of science [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP], includes over 1, 200 undergraduates in six departments of [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] study – biology, chemistry, mathematics, physics, pre - [SEP]
[CLS] how many years long was 

In [147]:
# third method , it is the most efficient
inputs = tokenizer(
    question,
    context,
    max_length=30,
    truncation="only_second",
    stride = 5,
    return_overflowing_tokens=True,

)
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the college of science was established at the university in 1865 by [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the university in 1865 by president father patrick dillon. dillon'[SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] patrick dillon. dillon's scientific courses were six years of [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] courses were six years of work, including higher - level mathematics [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] including higher - level mathematics courses. today the college, housed [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] today the college, housed in the newly built jordan hall of [SEP]
[CLS] how many 

In [148]:
# third method , it is the most efficient
inputs = tokenizer(
    question,
    context,
    max_length=60,
    truncation="only_second",
    stride = 25,
    return_overflowing_tokens=True,

)
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] the college of science was established at the university in 1865 by president father patrick dillon. dillon's scientific courses were six years of work, including higher - level mathematics courses. today the college, housed in the [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP] dillon's scientific courses were six years of work, including higher - level mathematics courses. today the college, housed in the newly built jordan hall of science, includes over 1, 200 undergraduates in six departments [SEP]
[CLS] how many years long was a scientific course under patrick dillon at notre dame? [SEP]. today the college, housed in the newly built jordan hall of science, includes over 1, 200 undergraduates in six departments of study – biology, chemistry, mathematics, physics, pre - professional studies, and [SEP]
[CLS] how many years long was a scientific

In [149]:
inputs = tokenizer(
    question,
    context,
    max_length=60,
    truncation="only_second",
    stride = 25,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,

)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [150]:
for i, offset in enumerate(inputs["offset_mapping"]):
  print(i,offset)

0 [(0, 0), (0, 3), (4, 8), (9, 14), (15, 19), (20, 23), (24, 25), (26, 36), (37, 43), (44, 49), (50, 57), (58, 64), (65, 67), (68, 73), (74, 78), (78, 79), (0, 0), (0, 3), (4, 11), (12, 14), (15, 22), (23, 26), (27, 38), (39, 41), (42, 45), (46, 56), (57, 59), (60, 64), (65, 67), (68, 77), (78, 84), (85, 92), (93, 99), (99, 100), (101, 107), (107, 108), (108, 109), (110, 120), (121, 128), (129, 133), (134, 137), (138, 143), (144, 146), (147, 151), (151, 152), (153, 162), (163, 169), (169, 170), (170, 175), (176, 187), (188, 195), (195, 196), (197, 202), (203, 206), (207, 214), (214, 215), (216, 222), (223, 225), (226, 229), (0, 0)]
1 [(0, 0), (0, 3), (4, 8), (9, 14), (15, 19), (20, 23), (24, 25), (26, 36), (37, 43), (44, 49), (50, 57), (58, 64), (65, 67), (68, 73), (74, 78), (78, 79), (0, 0), (101, 107), (107, 108), (108, 109), (110, 120), (121, 128), (129, 133), (134, 137), (138, 143), (144, 146), (147, 151), (151, 152), (153, 162), (163, 169), (169, 170), (170, 175), (176, 187), (188

In [151]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0, 0, 0]

In [152]:
inputs = tokenizer(
    squad_ds["train"][4:6]["question"],
    squad_ds["train"][4:6]["context"],
    max_length=60,
    truncation="only_second",
    stride = 25,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,

)
print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: \n{inputs['overflow_to_sample_mapping']}")

The 3 examples gave 26 features.
Here is where each comes from: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [153]:
len(inputs['input_ids'])

26

In [154]:
print(inputs.sequence_ids(0))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [155]:
answers = squad_ds["train"][4:6]["answers"]
answers

[{'text': ['Jay Z'], 'answer_start': [48]},
 {'text': ['six years'], 'answer_start': [134]}]

In [156]:
answers = squad_ds["train"][4:6]["answers"]

start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):

    print(i)
    # print(offset)

    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]

    print(answer)

    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])

    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    # Context tokens have sequence_id of 1
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

0
{'text': ['Jay Z'], 'answer_start': [48]}
1
{'text': ['Jay Z'], 'answer_start': [48]}
2
{'text': ['Jay Z'], 'answer_start': [48]}
3
{'text': ['Jay Z'], 'answer_start': [48]}
4
{'text': ['Jay Z'], 'answer_start': [48]}
5
{'text': ['Jay Z'], 'answer_start': [48]}
6
{'text': ['Jay Z'], 'answer_start': [48]}
7
{'text': ['Jay Z'], 'answer_start': [48]}
8
{'text': ['Jay Z'], 'answer_start': [48]}
9
{'text': ['Jay Z'], 'answer_start': [48]}
10
{'text': ['Jay Z'], 'answer_start': [48]}
11
{'text': ['Jay Z'], 'answer_start': [48]}
12
{'text': ['Jay Z'], 'answer_start': [48]}
13
{'text': ['Jay Z'], 'answer_start': [48]}
14
{'text': ['Jay Z'], 'answer_start': [48]}
15
{'text': ['Jay Z'], 'answer_start': [48]}
16
{'text': ['Jay Z'], 'answer_start': [48]}
17
{'text': ['Jay Z'], 'answer_start': [48]}
18
{'text': ['Jay Z'], 'answer_start': [48]}
19
{'text': ['six years'], 'answer_start': [134]}
20
{'text': ['six years'], 'answer_start': [134]}
21
{'text': ['six years'], 'answer_start': [134]}
22
{'

([32,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  40,
  23,
  0,
  0,
  0,
  0,
  0],
 [33,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  41,
  24,
  0,
  0,
  0,
  0,
  0])

In [157]:
idx = 5 # where ans is not there, only cls will be returned
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]

labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Jay Z, labels give: [CLS]


In [158]:
answers[sample_idx]["text"][0]

'Jay Z'

In [159]:
idx = 0 # where answer is there
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]

labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Jay Z, labels give: jay z


In [160]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = 384,
        truncation = "only_second",
        return_offsets_mapping = True,
        padding = "max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")

    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [161]:
squad_ds["train"].column_names

['id', 'title', 'context', 'question', 'answers']

In [162]:
tokenized_squad = squad_ds.map(preprocess_function, batched=True,
                               remove_columns=squad_ds["train"].column_names)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [163]:
tokenized_squad

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 100
    })
})

In [164]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

In [166]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 2

batches_per_epoch = len(tokenized_squad["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0,
                                       weight_decay_rate=0.01,
                                       num_train_steps=total_train_steps)


In [167]:
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [168]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [169]:
import tensorflow as tf
model.compile(optimizer=optimizer)
model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66364418 (253.16 MB)
Trainable params: 66364418 (253.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [170]:
model.fit(
    x = tf_train_set,
    validation_data = tf_validation_set,
    epochs = 2,
)

Epoch 1/2
25/25 [==============================] - 1258s 50s/step - loss: 5.5432 - val_loss: 5.1023
Epoch 2/2
25/25 [==============================] - 1168s 47s/step - loss: 4.8361 - val_loss: 4.7594


In [171]:
# Save your trained TensorFlow model to a local directory
model.save_pretrained("my_qna_model")
tokenizer.save_pretrained('my_qna_model')

('my_qna_model/tokenizer_config.json',
 'my_qna_model/special_tokens_map.json',
 'my_qna_model/vocab.txt',
 'my_qna_model/added_tokens.json',
 'my_qna_model/tokenizer.json')

In [172]:
#load the model and tokenizer
from transformers import TFAutoModelForQuestionAnswering, AutoTokenizer
model = TFAutoModelForQuestionAnswering.from_pretrained("my_qna_model")
tokenizer = AutoTokenizer.from_pretrained('my_qna_model')

Some layers from the model checkpoint at my_qna_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at my_qna_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [173]:
question = "Who is credited with the discovery of gravity?"

context = "In the 20th century, Albert Einstein revolutionized our understanding of gravity with his theory of general relativity.\
           This theory describes gravity as the curvature of spacetime caused by mass and energy."

In [174]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
question_answerer(question=question, context=context)

{'score': 0.005589987616986036,
 'start': 7,
 'end': 36,
 'answer': '20th century, Albert Einstein'}

In [175]:
inputs = tokenizer(question, context, return_tensors="tf")
outputs = model(**inputs)

In [176]:
outputs

TFQuestionAnsweringModelOutput(loss=None, start_logits=<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[ 0.4101945 ,  0.6542983 ,  0.5714765 ,  0.9190764 ,  0.9439925 ,
         0.7791149 ,  1.2200226 ,  0.8150323 ,  1.4087914 ,  0.57204574,
        -0.66918546,  0.85695106,  1.1137687 ,  1.9635113 ,  1.5988189 ,
        -0.4390574 ,  1.884322  ,  1.3320826 ,  0.85683984,  0.77299535,
         0.72850853,  1.0900155 ,  0.71677524,  1.3335255 ,  0.82568914,
         0.9072421 ,  1.3097562 ,  0.8137751 ,  1.6979145 ,  1.5961763 ,
        -0.73507553,  0.5566587 ,  0.8623471 ,  0.42506915,  1.2890586 ,
         0.5050928 ,  0.5861693 ,  1.1470959 ,  0.6495249 ,  1.3768531 ,
         1.3618654 ,  0.5691491 ,  0.8564247 ,  1.2489611 ,  0.3422857 ,
         1.0292348 , -0.70922476, -0.66922855]], dtype=float32)>, end_logits=<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[ 0.33896476,  0.30116376,  0.02698703,  0.49544767,  0.356691  ,
        -0.05384217,  0.7609437 ,  0.073

In [178]:
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis= -1)[0])
print(answer_start_index)
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis= -1)[0])
print(answer_end_index)

13
17


In [179]:
inputs.input_ids

<tf.Tensor: shape=(1, 48), dtype=int32, numpy=
array([[  101,  2040,  2003,  5827,  2007,  1996,  5456,  1997,  8992,
         1029,   102,  1999,  1996,  3983,  2301,  1010,  4789, 15313,
         4329,  3550,  2256,  4824,  1997,  8992,  2007,  2010,  3399,
         1997,  2236, 20805,  1012,  2023,  3399,  5577,  8992,  2004,
         1996, 25045,  1997,  2686,  7292,  3303,  2011,  3742,  1998,
         2943,  1012,   102]], dtype=int32)>

In [182]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'20th century, albert einstein'